# Definice

In [16]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

In [17]:
pd.set_option('display.max_columns', None)
# Definujte připojení k PostgreSQL
postgres_user = 'postgres'
postgres_password = 'MendeluRulez'
postgres_host = 'localhost'
postgres_port = '5432'
postgres_db = 'hulapp_v001'

In [18]:
# Vytvořte SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_db}')

In [19]:
data_dir = 'c:/spaces/pythons/mnd_spi_app/data/SLP_VYVOJ/vstup/csv/'


In [20]:
uid = {}
sql_query = "SELECT uid::character varying(255) FROM happ.firm where id_number = '62156489'"

# Načtěte data do DataFrame
df = pd.read_sql(sql_query, engine)

#print(df.head())

uid['firm'] = df.at[0, 'uid']
sql_query = f"SELECT uid::character varying(255) FROM happ.lhc where uid_firm = '{uid['firm']}'"
df = pd.read_sql(sql_query, engine)

#print(df.head())

uid['lhc'] = df.at[0, 'uid']

sql_query = f"SELECT uid::character varying(255) FROM happ.lhp where uid_firm = '{uid['firm']}' and uid_lhc = '{uid['lhc']}'"
df = pd.read_sql(sql_query, engine)

#print(df.head())

uid['lhp'] = df.at[0, 'uid']

sql_query = f"SELECT uid::character varying(255) FROM dsuhul.ik where uid_firm = '{uid['firm']}' and uid_lhc = '{uid['lhc']}' and uid_lhp = '{uid['lhp']}'"
df = pd.read_sql(sql_query, engine)

#print(df.head())
uid['ik'] = df.at[0, 'uid']


uid

{'firm': '9f4f5925-7a6c-4668-8af2-5c11e7343499',
 'lhc': '522560ba-ddcd-4436-8b0d-6f54c20a905f',
 'lhp': 'f5fec798-63cc-4fe0-b99c-2a59a1a3f74a',
 'ik': '5216750e-4c95-46d5-a2c5-ec2063d2d7d2'}

# IP

In [26]:
sql_query = f"""
 SELECT
    il.lokalita,
	ip.plocha::smallint,
	ip.datum_m,
	ip.status::character varying(255),
	ip.prist::character varying(255),
	ip.stab::character varying(255),
	ip.azimut_stab,
	ip.vzd_stab,
	ip.identifikace::character varying(255),
	ip.duv_neob::character varying(255),
	ip.kat_poz::character varying(255),
	ip.poz_les::character varying(255),
	ip.meric,
    COALESCE(ip.pozn_plocha, ''::character varying) AS pozn_plocha,
	ip.datum_m_pred,
	ip.prist_pred::character varying(255),
	ip.kat_poz_pred::character varying(255),
	ip.poz_les_pred::character varying(255),
    ip.nadm_vyska,
	ip.lt::character varying(255),
	ip.cil_zasoba,
	ip.cbp,
	ip.vyr_doba,
	ip.mdcel
   FROM dsuhul.ip ip
	, dsuhul.il il
  WHERE ip.uid_firm = '{uid['firm']}'::uuid AND ip.uid_lhc = '{uid['lhc']}'::uuid AND ip.uid_lhp = '{uid['lhp']}'::uuid
      AND il.uid = ip.uid_il
  ORDER BY (il.lokalita::integer)
"""
df_ip = pd.read_sql(sql_query, engine)
df_ip.to_csv(data_dir + 'ip.csv', decimal=',', sep=';', index=False)
df_ip

,lokalita,plocha,datum_m,status,prist,stab,azimut_stab,vzd_stab,identifikace,duv_neob,kat_poz,poz_les,meric,pozn_plocha,datum_m_pred,prist_pred,kat_poz_pred,poz_les_pred,nadm_vyska,lt,cil_zasoba,cbp,vyr_doba,mdcel
0,10000001.0,1,2022-05-25,100,100,100,0.0,0.0,100,,100,100,Zahradnicek,,2013-07-08,100,100,100,331.60,2S4,150.0,5.8,50.0,4.6
1,10000002.0,1,2022-06-06,100,100,100,0.0,0.0,100,,100,100,Zahradnicek,,2013-07-08,100,100,100,309.95,2S1,150.0,5.8,50.0,4.6
2,10000003.0,1,2022-06-07,100,100,100,0.0,0.0,100,,100,100,Zahradnicek,,2013-07-08,100,100,100,282.68,2C1,150.0,5.8,50.0,4.6
3,10000012.0,1,2022-06-10,100,100,100,0.0,0.0,100,,100,100,Zahradnicek,,2013-07-15,100,100,100,351.38,2B4,150.0,5.8,50.0,4.6
4,10000013.0,1,2022-06-10,100,100,100,0.0,0.0,100,,100,100,Zahradnicek,,2013-07-15,100,100,100,338.23,2B4,150.0,5.8,50.0,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,140584850.0,1,2022-12-27,200,100,100,0.0,0.0,,,100,100,Zahradnicek,,1901-01-01,,,,315.10,3J2,250.0,5.8,50.0,4.6
1203,140625052.0,1,2022-12-28,200,100,100,0.0,0.0,,,100,100,Zahradnicek,,1901-01-01,,,,446.16,3J8,250.0,6.8,50.0,4.6
1204,140629081.0,1,2022-12-28,200,100,100,0.0,0.0,,,100,100,Zahradnicek,,1901-01-01,,,,437.46,3X1,250.0,3.5,50.0,4.6
1205,140632058.0,1,2022-12-28,200,100,100,0.0,0.0,,,100,100,Zahradnicek,,1901-01-01,,,,395.61,3J8,250.0,5.8,50.0,4.6


# VZ

In [32]:
sql_query = f"""
  SELECT
    il.lokalita::integer,
    ip.plocha::integer,
    vz.kmen::integer,
    COALESCE(vz.dr_zkr, ''::character varying) AS dr_zkr,
    vz.x_m,
    vz.y_m,
    vz.azimut_km,
    vz.vzd_km,
    COALESCE(vz.opak_ident_km, ''::character varying) AS opak_ident_km,
    COALESCE(vz.parez, ''::character varying) AS parez,
    COALESCE(vz.pol_parez, ''::character varying) AS pol_parez,
    COALESCE(vz.vyklizeni_km, ''::character varying) AS vyklizeni_km,
    COALESCE(vz.sous, ''::character varying) AS sous,
    COALESCE(vz.zlom_vyvrat, ''::character varying) AS zlom_vyvrat,
    COALESCE(vz.dvojak, ''::character varying) AS dvojak,
    COALESCE(vz.kmen_pred, 0::numeric)::integer AS kmen_pred,
    COALESCE(vz.parez_pred, ''::character varying) AS parez_pred,
    COALESCE(vz.sous_pred, ''::character varying) AS sous_pred,
    COALESCE(vz.zlom_vyvrat_pred, ''::character varying) AS zlom_vyvrat_pred,
    COALESCE(vz.tloustka_km, 0::numeric) AS tloustka_km,
    COALESCE(vz.vyska, 0::numeric) AS mvyska,
    COALESCE(vz.mod_vys, 0::numeric) AS mod_vys,
    COALESCE(vz.tloustka_km_pred, 0::numeric) AS tloustka_km_pred,
    COALESCE(vz.vyska_pred, 0::numeric) AS mvyska_pred,
    COALESCE(vz.mod_vys_pred, 0::numeric) AS mod_vys_pred,
    COALESCE(vz.pno, 0::numeric) AS pno,
    COALESCE(vz.pno_pred, 0::numeric) AS pno_pred,
	CASE
		WHEN COALESCE(vz.vzornik2, false) THEN 'A'::text
		ELSE 'N'::text
	END AS vzornik2,
	CASE
		WHEN COALESCE(vz.vzornik2_pred, false) THEN 'A'::text
		ELSE 'N'::text
	END AS vzornik2_pred,
    COALESCE(vz.d13_depl, 0::numeric) AS d13_depl,
    COALESCE(vz.mod_vys_depl, 0::numeric) AS mod_vys_depl,
    COALESCE(vz.pozn_km, ''::character varying) AS pozn_km
   FROM dsuhul.ip ip,
    dsuhul.il il,
    dsuhul.vz vz
  WHERE ip.uid_firm = '{uid['firm']}'::uuid AND ip.uid_lhc = '{uid['lhc']}'::uuid AND ip.uid_lhp = '{uid['lhp']}'::uuid
      AND il.uid = ip.uid_il AND vz.uid_ip = ip.uid
      --AND vz.zapocitat
  ORDER BY il.lokalita, vz.kmen, vz.kmen_pred;
"""
df_vz = pd.read_sql(sql_query, engine)
df_vz.to_csv(data_dir + 'vz.csv', decimal=',', sep=';', index=False)
df_vz

,lokalita,plocha,kmen,dr_zkr,x_m,y_m,azimut_km,vzd_km,opak_ident_km,parez,pol_parez,vyklizeni_km,sous,zlom_vyvrat,dvojak,kmen_pred,parez_pred,sous_pred,zlom_vyvrat_pred,tloustka_km,mvyska,mod_vys,tloustka_km_pred,mvyska_pred,mod_vys_pred,pno,pno_pred,vzornik2,vzornik2_pred,d13_depl,mod_vys_depl,pozn_km
0,10000001,1,1,BO,-0.11,2.65,357.6,2.65,,100,100,100,100,100,100,1,100,,,35.4,24.8,22.5,32.9,19.8,21.9,0.0,0.0,A,A,0.0,0.0,
1,10000001,1,2,BO,-5.71,10.56,331.6,12.00,,400,100,100,100,100,100,2,100,,,33.1,0.0,22.0,31.9,0.0,21.7,0.0,0.0,N,N,0.0,0.0,
2,10000001,1,3,HB,-4.49,5.34,319.9,6.98,,100,100,,100,,100,3,100,,,20.7,0.0,15.5,17.9,12.0,14.3,0.0,0.0,N,N,0.0,0.0,
3,10000001,1,4,DBZ,-0.84,6.02,352.1,6.08,,100,100,,500,400,100,4,100,100,100,16.3,8.0,12.7,16.6,15.9,12.9,85.0,0.0,N,N,0.0,0.0,
4,10000001,1,5,LP,2.83,6.03,25.1,6.66,,100,100,,100,100,100,5,100,,,15.6,0.0,13.6,14.9,9.9,13.2,0.0,0.0,N,N,0.0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17661,140633918,1,8,JV,-1.89,0.43,282.8,1.94,,100,,,100,100,100,0,,,,15.8,10.8,11.3,0.0,0.0,0.0,0.0,0.0,A,N,0.0,0.0,
17662,140633918,1,9,BO,0.70,1.98,19.5,2.10,,100,,,100,100,100,0,,,,49.3,14.7,16.9,0.0,0.0,0.0,0.0,0.0,A,N,0.0,0.0,
17663,140633918,1,10,BO,0.49,6.37,4.4,6.39,,100,,,100,100,100,0,,,,47.0,15.6,16.8,0.0,0.0,0.0,0.0,0.0,N,N,0.0,0.0,h
17664,140633918,1,11,AK,-3.70,1.80,295.9,4.11,,100,,,100,100,100,0,,,,12.5,0.0,12.2,0.0,0.0,0.0,0.0,0.0,N,N,0.0,0.0,


# PNO

In [27]:
sql_query = f"""
	select
		lokalita
		, 1 as plocha
		, pomno
		, x_m
		, y_m
		, popis
	from sandbox.pno_imp
	order by lokalita::integer, pomno
"""
df_pno = pd.read_sql(sql_query, engine)
df_pno.to_csv(data_dir + 'pno.csv', decimal=',', sep=';', index=False)
df_pno

,lokalita,plocha,pomno,x_m,y_m,popis
0,10000001.0,1,1.0,13.95,8.16,LP DBH=120mm
1,10000001.0,1,2.0,-3.32,15.33,HB DBH=130mm
2,10000002.0,1,1.0,-17.67,4.72,DB DBH=150mm
3,10000002.0,1,2.0,8.97,11.67,
4,10000003.0,1,1.0,-22.14,-13.52,
...,...,...,...,...,...,...
2422,140629081.0,1,2.0,-9.90,-10.36,JV DBH 286
2423,140632058.0,1,1.0,7.36,-13.44,BK DBH 345
2424,140632058.0,1,2.0,-6.44,-9.55,BK DBH 93
2425,140633918.0,1,1.0,5.86,-8.25,HB 13 cm


# Modelové výšky

## Aktuální inventarizace

In [36]:
sql_query = f"""
  SELECT
    il.lokalita::integer,
    ip.plocha::integer,
    vz.kmen::integer,
    COALESCE(vz.mod_vys, 0::numeric) AS mod_vys
   FROM dsuhul.ip ip,
    dsuhul.il il,
    dsuhul.vz vz
  WHERE ip.uid_firm = '{uid['firm']}'::uuid AND ip.uid_lhc = '{uid['lhc']}'::uuid AND ip.uid_lhp = '{uid['lhp']}'::uuid
      AND il.uid = ip.uid_il AND vz.uid_ip = ip.uid
      AND vz.kmen > 0
	  AND COALESCE(vz.mod_vys, 0::numeric) > 0
  ORDER BY il.lokalita, vz.kmen, vz.kmen_pred;
"""
df_mod = pd.read_sql(sql_query, engine)
df_mod.to_csv(data_dir + 'mod_vys.csv', decimal=',', sep=';', index=False)
df_mod

,lokalita,plocha,kmen,mod_vys
0,10000001,1,1,22.5
1,10000001,1,2,22.0
2,10000001,1,3,15.5
3,10000001,1,4,12.7
4,10000001,1,5,13.6
...,...,...,...,...
14517,140633918,1,8,11.3
14518,140633918,1,9,16.9
14519,140633918,1,10,16.8
14520,140633918,1,11,12.2


## Předchozí inventarizace

In [38]:
sql_query = f"""
  SELECT
    il.lokalita::integer,
    ip.plocha::integer,
    vz.kmen_pred::integer,
    COALESCE(vz.mod_vys_pred, 0::numeric) AS mod_vys_pred
   FROM dsuhul.ip ip,
    dsuhul.il il,
    dsuhul.vz vz
  WHERE ip.uid_firm = '{uid['firm']}'::uuid AND ip.uid_lhc = '{uid['lhc']}'::uuid AND ip.uid_lhp = '{uid['lhp']}'::uuid
      AND il.uid = ip.uid_il AND vz.uid_ip = ip.uid
      AND vz.kmen_pred > 0
	  AND COALESCE(vz.mod_vys_pred, 0::numeric) > 0
  ORDER BY il.lokalita, vz.kmen, vz.kmen_pred;
"""
df_mod_pred = pd.read_sql(sql_query, engine)
df_mod_pred.to_csv(data_dir + 'mod_vys_pred.csv', decimal=',', sep=';', index=False)
df_mod_pred

,lokalita,plocha,kmen_pred,mod_vys_pred
0,10000001,1,1,21.9
1,10000001,1,2,21.7
2,10000001,1,3,14.3
3,10000001,1,4,12.9
4,10000001,1,5,13.2
...,...,...,...,...
4515,100000255,1,5,32.4
4516,100000255,1,6,28.8
4517,100000255,1,7,17.7
4518,100000255,1,8,23.0
